In [19]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks, Model, layers, backend as K
import SpeechModels
from Project2_LSTM.augment_layers import FreqMaskLayer, TimeMaskLayer


In [37]:
tf.config.list_logical_devices()
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200
TEST_PATH = "tensorflow-speech-recognition-challenge/test"
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [29]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True
)


label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [38]:
test_dataset = tf.keras.utils.audio_dataset_from_directory(
    directory=TEST_PATH,
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

Found 158538 files belonging to 1 classes.


In [25]:
test_dataset.class_names

['audio']

In [27]:
dataset = train_ds.concatenate(val_ds).concatenate(test_ds).rebatch(512)
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

dataset = dataset.map(squeeze, tf.data.AUTOTUNE)
test_dataset = test_dataset.map(squeeze, tf.data.AUTOTUNE)

def create_model(freq=False, time=False):
    m = SpeechModels.get_melspec_model(iLen=16000)
    m.trainable = False
    inputs, outputs = m.inputs, m.outputs

    x = m(inputs)
    if freq:
        x = FreqMaskLayer(10)(x)
    if time:
        x = TimeMaskLayer(10)(x)
    x = tf.expand_dims(x, axis=-1, name='mel_stft')

    x = layers.Conv2D(10, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    # x = Reshape((125, 80)) (x)
    # keras.backend.squeeze(x, axis)
    x = layers.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                             )(x)  # [b_s, seq_len, vec_dim]
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True)
                             )(x)  # [b_s, seq_len, vec_dim]

    x_first = layers.Lambda(lambda q: q[:, -1])(x)  # [b_s, vec_dim]
    query = layers.Dense(128)(x_first)

    # dot product attention
    att_scores = layers.Dot(axes=[1, 2])([query, x])
    att_scores = layers.Softmax(name='attSoftmax')(att_scores)  # [b_s, seq_len]

    # rescale sequence
    att_vector = layers.Dot(axes=[1, 1])([att_scores, x])  # [b_s, vec_dim]
    x = layers.Dropout(rate=0.3)(att_vector)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(rate=0.3)(x)
    x = layers.Dense(32)(x)
    x = layers.Dropout(rate=0.3)(x)
    output = layers.Dense(N_CLASS, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = create_model(freq=True)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 normalized_spectrogram_model (  (None, 125, 80)     0           ['input[0][0]']                  
 Functional)                                                                                      
                                                                                                  
 freq_mask_layer (FreqMaskLayer  (None, 125, 80)     0           ['normalized_spectrogram_model[0]
 )                                                               [0]']                            
                                                                                            

In [23]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)
model_checkpoint = callbacks.ModelCheckpoint(
    filepath="best_model_checkpoint",
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [ ]:
history = model.fit(
    dataset,
    epochs=MAX_EPOCHS,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

In [30]:
predictions = model.predict(test_dataset)

1/1 [==============================] - 12s 12s/step


In [36]:
predict_class = np.array(list(label_names[i] for i in np.argmax(predictions, axis=1)))

array(['up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'unknown', 'up', 'unknown', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'unknown', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'unknown',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'unknown', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'unknown', 'up',
       'up', 'up', 'up', 'up', 'off', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', '

In [48]:
for i, (root, dir, fname) in enumerate(os.walk(os.path.join(TEST_PATH, 'audio'))):
    files = np.array(fname)

In [54]:
submission = pd.DataFrame(np.vstack([files, predict_class]).T)
submission.to_csv("submission.csv")